In [9]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
from numba import jit
import warnings

In [10]:
def rossler(xyz, *, a=0.2, b=0.2, c=5.7):
    x, y, z = xyz
    x_dot = -y-z
    y_dot = x + a*y
    z_dot = b + z*(x-c)
    return np.array([x_dot, y_dot, z_dot])

def rossler_dm_y(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = y
    y_dot = x + a*y
    z_dot = -y - z + a*x + a*a*y
    return np.array([x_dot, y_dot, z_dot])

def rossler_dm_x(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = x
    y_dot = -y-z
    z_dot = -x-a*y-b-z*(x-c)  
    return np.array([x_dot, y_dot, z_dot])

def rossler_dm_z(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = z
    y_dot = b + z*(x-c)
    z_dot = (b + z*(x-c))*(x-c) + z*(-y-z)
    return np.array([x_dot, y_dot, z_dot])

def rossler_dm_yz(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = y+z
    y_dot = b+x+a*y-c*z+x*z
    z_dot = -b*c + (a+b)*x + (a*a-1)*y + (c*c - 1)*z - (2*c + 1)*x*z - z*z + x*x*z
    return np.array([x_dot, y_dot, z_dot])

def rossler_dm_zx(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = z + x
    y_dot = -y - z + b + z*(x-c)
    z_dot = -b*(c+1) + (b-1)*x - a*y + c*(c+1)*z + (1-2*c)*x*z - y*z - z*z
    return np.array([x_dot, y_dot, z_dot])

def rossler_dm_xy(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = x + y
    y_dot = x + (a-1)*y - z
    z_dot = -b + (a-1)*x + (a*a - a + 1)*y + (c-1)*z - x*z
    return np.array([x_dot, y_dot, z_dot])

def rossler_y_z_y(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = y
    y_dot = z 
    z_dot = x + a*y
    return np.array([x_dot, y_dot, z_dot])
def rossler_x_y_x(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = x
    y_dot = y 
    z_dot = -y-z
    return np.array([x_dot, y_dot, z_dot])
def rossler_x_y_y(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = x
    y_dot = y 
    z_dot = x+a*y
    return np.array([x_dot, y_dot, z_dot])
def rossler_y_z_z(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = y
    y_dot = z 
    z_dot = b + z*(x-c)
    return np.array([x_dot, y_dot, z_dot])
def rossler_x_z_z(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = x
    y_dot = z 
    z_dot = b + z*(x-c)
    return np.array([x_dot, y_dot, z_dot])
def rossler_x_z_x(x,y,z, a = 0.2, b = 0.2, c = 5.7):
    x_dot = x
    y_dot = z 
    z_dot = -y-z
    return np.array([x_dot, y_dot, z_dot])

In [11]:
@jit(nopython=True, fastmath=True)
def compute_errors_numba(neighbor_clouds, max_iter=50, eps=1e-5):
    n_samples, k, dim = neighbor_clouds.shape
    errors = np.empty(n_samples, dtype=np.float64)
    
    for i in range(n_samples):
        cloud = neighbor_clouds[i]
        
        y = np.zeros(dim)
        for d in range(dim):
            sum_val = 0.0
            for j in range(k):
                sum_val += cloud[j, d]
            y[d] = sum_val / k
            
        for _ in range(max_iter):
            sum_weights = 0.0
            y_next = np.zeros(dim)
            distances = np.empty(k)
            all_non_zero = True
            
            for j in range(k):
                dist_sq = 0.0
                for d in range(dim):
                    diff = cloud[j, d] - y[d]
                    dist_sq += diff * diff
                dist = np.sqrt(dist_sq)
                distances[j] = dist
                
                if dist < 1e-10: 
                    all_non_zero = False
                    for d in range(dim): y[d] = cloud[j, d]
                    break
                
                w = 1.0 / dist
                sum_weights += w
                for d in range(dim):
                    y_next[d] += cloud[j, d] * w
            
            if not all_non_zero: break 
            
            diff_norm_sq = 0.0
            for d in range(dim):
                y_next[d] /= sum_weights
                diff = y[d] - y_next[d]
                diff_norm_sq += diff * diff
                y[d] = y_next[d]
                
            if np.sqrt(diff_norm_sq) < eps:
                break
        
        total_dist = 0.0
        for j in range(k):
            dist_sq = 0.0
            for d in range(dim):
                diff = cloud[j, d] - y[d]
                dist_sq += diff * diff
            total_dist += np.sqrt(dist_sq)
            
        errors[i] = total_dist / k
        
    return errors

def check_embedding_condition(data, threshold=1e10):
    data_centered = data - np.mean(data, axis=0)
    s = np.linalg.svdvals(data_centered)
    if s[-1] < 1e-12:
        return np.inf
    return s[0] / s[-1]

def power_law_model(k, intercept, coef, gamma):
    return intercept + coef * (k ** gamma)

# Estimate intrinsic stochasticity at fixed k
import numpy as np
import warnings
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import NearestNeighbors

def estimate_intrinsic_stochasticity(
    embedding_data,
    pushforward,
    k=50,
    n_samples=5000,
    cond_threshold=1000,
    random_state=0,
    return_pointwise=False,
    theiler_w=0,          
    buffer_mult=6,        
    pre_scaled=False,     
    strict_theiler=True,  
):
    if k <= 1:
        raise ValueError("k must be >= 2.")
    if pushforward <= 0:
        raise ValueError("pushforward must be >= 1.")
    if theiler_w < 0:
        raise ValueError("theiler_w must be >= 0.")
    if buffer_mult < 1:
        raise ValueError("buffer_mult must be >= 1.")

    if pre_scaled:
        data_norm = np.asarray(embedding_data)
    else:
        scaler = RobustScaler()
        data_norm = scaler.fit_transform(np.asarray(embedding_data))

    cond_num = check_embedding_condition(data_norm)
    if (cond_num > cond_threshold) or np.isinf(cond_num):
        warnings.warn(
            f"CRITICAL WARNING: Rank Deficient! Condition Number = {cond_num:.2e}. Skipping."
        )
        return None

    T = data_norm.shape[0]
    max_start = T - pushforward
    if max_start <= 0:
        raise ValueError("Pushforward too large for the given data length.")

    X_curr = data_norm[:max_start]
    Y_fut  = data_norm[pushforward:]

    rng = np.random.default_rng(random_state)
    n_q = int(min(n_samples, max_start))
    query_idx = rng.choice(max_start, size=n_q, replace=False)

    k_query = int(min(max_start, max(k * buffer_mult, k + 2 * theiler_w + 5)))
    nbrs = NearestNeighbors(n_neighbors=k_query, algorithm="auto", n_jobs=-1).fit(X_curr)

    dist_raw, idx_raw = nbrs.kneighbors(X_curr[query_idx]) 

    nn_idx = np.empty((n_q, k), dtype=np.int64)
    rk = np.empty(n_q, dtype=np.float64)  

    not_enough = 0

    for r in range(n_q):
        q = int(query_idx[r])
        cand = idx_raw[r]
        cand_d = dist_raw[r]

        picked = 0
        last_d = 0.0

        for j in range(k_query):
            t = int(cand[j])
            if t == q:
                continue
            if theiler_w > 0 and (abs(t - q) <= theiler_w):
                continue
            nn_idx[r, picked] = t
            last_d = float(cand_d[j])  
            picked += 1
            if picked == k:
                break

        if picked < k:
            not_enough += 1
            if strict_theiler:
                raise RuntimeError(
                    f"[Theiler window] Not enough valid neighbors after exclusion: "
                    f"query_index={q}, picked={picked} < k={k}. "
                    f"Try increasing buffer_mult (current={buffer_mult}) or decreasing theiler_w (current={theiler_w}), "
                    f"or reducing k / n_samples."
                )
            else:
                picked2 = 0
                last_d = 0.0
                for j in range(k_query):
                    t = int(cand[j])
                    if t == q:
                        continue
                    nn_idx[r, picked2] = t
                    last_d = float(cand_d[j])
                    picked2 += 1
                    if picked2 == k:
                        break

        rk[r] = last_d

    neighbor_clouds = Y_fut[nn_idx]  # (n_q, k, d)

    local_errors = compute_errors_numba(neighbor_clouds)
    E_star_k = float(np.mean(local_errors))

    out = {
        "E_star_k": E_star_k,
        "k": int(k),
        "pushforward": int(pushforward),
        "theiler_w": int(theiler_w),
        "n_queries": int(n_q),
        "condition_number": float(cond_num),
        "not_enough_queries": int(not_enough),
        "strict_theiler": bool(strict_theiler),
        "median_rk": float(np.median(rk)),
        "q90_rk": float(np.quantile(rk, 0.90)),
        "mean_rk": float(np.mean(rk)),
    }
    if return_pointwise:
        out["pointwise_errors"] = local_errors
        out["query_idx"] = query_idx
        out["nn_idx"] = nn_idx
        out["rk"] = rk
    return out

In [ ]:
def build_embeddings_from_y(y_arr):
    x_x_x = rossler_dm_x(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    y_y_y = rossler_dm_y(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    z_z_z = rossler_dm_z(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    y_z_y = rossler_y_z_y(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    x_y_x = rossler_x_y_x(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    x_y_y = rossler_x_y_y(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    y_z_z = rossler_y_z_z(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    x_z_z = rossler_x_z_z(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    x_z_x = rossler_x_z_x(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    y_plus_z = rossler_dm_yz(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    x_plus_z = rossler_dm_zx(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T
    x_plus_y = rossler_dm_xy(y_arr[:,0], y_arr[:,1], y_arr[:,2]).T

    return {
        "x_x_x": x_x_x,
        "y_y_y": y_y_y,
        "z_z_z": z_z_z,
        "y_z_y": y_z_y,
        "x_y_x": x_y_x,
        "x_y_y": x_y_y,
        "y_z_z": y_z_z,
        "x_z_z": x_z_z,
        "x_z_x": x_z_x,
        "y_plus_z": y_plus_z,
        "x_plus_z": x_plus_z,
        "x_plus_y": x_plus_y,
    }

dt = 0.01
num_steps = int(2e4)

y_clean = np.zeros((num_steps + 1, 3), dtype=np.float64)
y_clean[0] = np.array([1.0, 1.0, 0.0], dtype=np.float64)

for i in range(num_steps):
    y_clean[i + 1] = y_clean[i] + rossler(y_clean[i]) * dt

embeddings_clean = build_embeddings_from_y(y_clean)

y_std = np.std(y_clean, axis=0, ddof=0)
y_scale = float(np.mean(y_std))

PUSHFORWARD = 20
K = 50
N_SAMPLES = 5000
COND_THRESHOLD = 1000
THEILER_W = 30
BUFFER_MULT = 6

noise_levels = [0.00, 0.01, 0.05, 0.10]
noise_seed = 123
n_repeats = 10

all_rows = []

for p in noise_levels:
    for rep in range(n_repeats):
        rng = np.random.default_rng(noise_seed + rep)

        for name, emb_clean in embeddings_clean.items():
            emb_arr = np.asarray(emb_clean)
            if emb_arr.ndim == 1:
                emb_arr = emb_arr.reshape(-1, 1)

            if p == 0.0:
                emb_noisy = emb_arr
            else:
                sigma = p * y_scale
                emb_noisy = emb_arr + rng.normal(0.0, sigma, size=emb_arr.shape)

            out = estimate_intrinsic_stochasticity(
                embedding_data=emb_noisy,
                pushforward=PUSHFORWARD,
                k=K,
                n_samples=N_SAMPLES,
                cond_threshold=COND_THRESHOLD,
                random_state=rep,
                return_pointwise=False,
                theiler_w=THEILER_W,
                buffer_mult=BUFFER_MULT,
                pre_scaled=False,
                strict_theiler=True,
            )

            if out is None:
                all_rows.append({
                    "noise_level": p,
                    "repeat": rep,
                    "embedding": name,
                    "E_star_k": np.nan,
                    "median_rk": np.nan,
                    "q90_rk": np.nan,
                    "status": "skipped(cond)",
                })
            else:
                all_rows.append({
                    "noise_level": p,
                    "repeat": rep,
                    "embedding": name,
                    "E_star_k": out["E_star_k"],
                    "median_rk": out["median_rk"],
                    "q90_rk": out["q90_rk"],
                    "status": "ok",
                })

df_raw = pd.DataFrame(all_rows)

def agg_mean_std(s):
    return pd.Series({"mean": float(np.nanmean(s)), "std": float(np.nanstd(s, ddof=0))})

summary = (
    df_raw.groupby(["noise_level", "embedding"])
          .agg(
              E_star_k_mean=("E_star_k", lambda s: float(np.nanmean(s))),
              E_star_k_std =("E_star_k", lambda s: float(np.nanstd(s, ddof=0))),
              median_rk_mean=("median_rk", lambda s: float(np.nanmean(s))),
              median_rk_std =("median_rk", lambda s: float(np.nanstd(s, ddof=0))),
              q90_rk_mean=("q90_rk", lambda s: float(np.nanmean(s))),
              q90_rk_std =("q90_rk", lambda s: float(np.nanstd(s, ddof=0))),
              n_ok=("status", lambda s: int(np.sum(np.array(s) == "ok"))),
          )
          .reset_index()
)

summary_disp = summary.copy()
for col in ["E_star_k_mean","E_star_k_std","median_rk_mean","median_rk_std","q90_rk_mean","q90_rk_std"]:
    summary_disp[col] = summary_disp[col].round(4)

print("\n=== Summary: E* and effective radii (mean±std over repeats) ===")
print(summary_disp.sort_values(["noise_level", "E_star_k_mean"], ascending=[True, True]))

print("\n=== Raw runs ===")
print(df_raw.head(20))